In [84]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

In [85]:
import spacy

# Load English tokenizer, tagger, parser, NER, and word vectors
spacy.cli.download('en_core_web_sm')
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [86]:
from pathlib import Path
import itertools

def read_problem_files(problem_folder, n=None):
    """
    reads ground truth files into dict
    :param truth_folder: path to folder holding ground truth files
    :return: dict of ground truth files with problem-id as key and file content as value
    """
    problems = {}
    files = itertools.islice(Path(problem_folder).glob('problem-*.txt'), n)
    for problem_file in files:
        number = problem_file.name[len("problem-") : -len(".txt")]
        with open(problem_file, 'r', encoding="utf8") as fh:
            problems[number] = fh.readlines()
    return problems

In [87]:
from evaluation.evaluator import read_ground_truth_files

ground_truth = read_ground_truth_files("pan21/train")

In [93]:
from sklearn.metrics.pairwise import cosine_similarity

def most_similar(para1, para2):
    embeddings1 = sbert_model.encode(para1, convert_to_numpy=True)
    embeddings2 = sbert_model.encode(para2, convert_to_numpy=True)

    # Calculate cosine similarity between all sentence pairs
    similarity_matrix = cosine_similarity(embeddings1, embeddings2)

    # Find the indices of the most similar sentence pair
    max_similarity_index = divmod(similarity_matrix.argmax(), similarity_matrix.shape[1])

    # Get the most similar sentence pair
    most_similar_pair = (para1[max_similarity_index[0]], para2[max_similarity_index[1]])

    return most_similar_pair, similarity_matrix


In [89]:
from transformers import BertTokenizer, BertModel
import torch

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [90]:
def get_word_embeddings(sentence):
    # Tokenize the sentence
    tokens = bert_tokenizer.tokenize(sentence)
    token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)

    # Convert token IDs to tensor
    input_ids = torch.tensor([token_ids])

    # Get BERT model output
    with torch.no_grad():
        outputs = bert_model(input_ids)

    # Extract word embeddings from BERT model output
    return outputs[0].numpy()

In [91]:
import numpy as np


def compare_sentences(sent1, sent2):
    diff = len(sent1) - len(sent2)
    embed1 = get_word_embeddings(sent1)
    embed2 = get_word_embeddings(sent2)

    if diff > 0:
        excess = np.sum(embed1[:-diff])/len(sent2)
        embed1 = [word + excess for word in embed1[:len(sent2)]]
    elif diff < 0:
        excess = np.sum(embed2[:-diff], axis=0)/len(sent1)
        embed2 = [word + excess for word in embed2[:len(sent1)]]

    mean_cos_sim = np.mean([cosine_similarity(w1, w2) for w1, w2 in zip(embed1, embed2)])

    return mean_cos_sim



In [94]:
# Our sentences to encode
problems = read_problem_files("pan21/train", n=4)

for num, problem in problems.items():
    problem_num = f"problem-{num}"
    paragraphs = [[sent.text for sent in nlp(paragraph).sents] for paragraph in problem]
    for i in range(len(paragraphs) - 1):
        para1, para2 = (paragraphs[i], paragraphs[i + 1])

        most_similar_pair, similarity = most_similar(para1, para2)
        highest_similarity = compare_sentences(*most_similar_pair)
        print(f"{i}, {i+1}: {similarity:.2f} {highest_similarity:.2f} {ground_truth[problem_num]['changes'][i]}")
    print(f"{ground_truth[problem_num]=}")

ValueError: Expected 2D array, got 1D array instead:
array=[ 7.85434470e-02  1.96468760e-03  3.33422869e-02  8.77521001e-03
 -4.53390777e-02 -4.55802456e-02 -7.99855664e-02  2.06299219e-02
 -4.89074476e-02  1.21216655e-01  4.43624742e-02 -1.71473436e-02
 -7.34435543e-02  1.75592247e-02  7.47129507e-03 -5.92222326e-02
  4.30323742e-02  1.35327969e-02 -4.57067341e-02 -4.03561480e-02
 -4.45564911e-02 -2.74355561e-02  4.07087915e-02 -4.56368327e-02
 -8.04605894e-03  4.93553914e-02 -4.01860364e-02  4.55614291e-02
 -1.91965618e-03 -4.65979287e-03  3.09489272e-03  5.99665269e-02
  8.00852664e-03 -8.86062346e-03 -4.91302721e-02  4.52211499e-02
  8.80126667e-04  2.64338590e-02 -5.85633405e-02 -3.89617234e-02
 -9.33800861e-02  3.04856151e-03  4.27806303e-02  7.59484544e-02
  5.80741279e-02  6.20034225e-02 -1.62492190e-02 -7.10250214e-02
 -4.57938202e-03 -1.61669217e-02 -5.50461598e-02 -6.02390580e-02
  9.49239917e-03 -2.60158647e-02  1.05940878e-01  4.16961201e-02
  1.44326501e-02 -4.01123660e-0